## 7.14 函数


函数调用可能会降低程序速度，原因如下：

- 函数调用会让微处理器跳转到不同的代码地址然后返回。这可能需要多达4个时钟周期。
在大多数情况下，微处理器能够将调用及返回操作与其他计算重叠运行以节省时间。

- 如果代码碎片化地分散在内存中，代码缓存的效率会降低。

- 函数参数以32位模式存储在堆栈中。
将参数存储在堆栈上并再次读取它们需要额外的时间。
如果某个参数处于关键依赖链条中，因此带来的延时是需要重视的。

- 设置堆栈帧，保存和恢复寄存器，以及保存异常处理信息（可能需要，也可能不需要），都需要额外的时间。

- 每个函数调用语句占用分支目标缓冲区（BTB）中的一个单元。
如果程序的关键部分有很多函数调用和分支，BTB中的争用可能导致分支预测错误

可以使用以下方法，来减少在关键程序的中花费在函数调用上的时间。

### 避免非必要的函数

**一些编程教科书建议每个长于若干行的函数应该分成多个函数。我不赞成此规则。**
将函数拆分为多个较小的函数只会降低程序的效率。
除非函数执行多个逻辑上不同的任务，否则仅仅因为它很长就拆分函数，并不会使程序更加清晰。
可能的话，关键性的最内层环应该完全放入一个函数内。

### 使用内联函数

内联函数可像宏一样扩展，以便调用函数的每个语句都被函数体替换。
如果使用了`inline`关键字或者在类定义中包含了函数体，则通常函数会被内联。
如果函数很小，或者仅从程序中的一个地方调用该函数，则内联函数是有好处的。
小函数通常由编译器自动内联。
另一方面，如果内联导致技术问题或性能问题，编译器在某些情况下可能会忽略内联函数的请求。

### 最内层循环避免函数嵌套

调用其他函数的函数称为**帧函数**，而不调用任何其他函数的函数称为**叶函数**。
叶函数比函数有更高效率，原因如第63页所述。
如果程序的关键部分的最内层循环包含对帧函数的调用，则可通过下面方法来改进代码：
- 内联帧函数
- 把帧函数转换为叶函数，转换方法：内联所有被调用的函数

### 使用宏代替函数

使用`#define`声明的宏肯定会被内联。但请注意，每次使用宏参数时都会对其进行评估。例如：
```cpp
// Example 7.34a. Use macro as inline function
#define MAX(a,b) (a > b ? a : b)
y = MAX(f(x), g(x));
```
在这个例子中, `f(x)` 或 `g(x)` 被计算了两次，原因是它被引用了两次。
你可以通过使用内联函数而不是用宏，来避免这种情况。如果您希望该内联函数支持任何类型的参数，那么将参数设计为模板：
```cpp
// Example 7.34b. Replace macro by template
template <typename T>
static inline T max(T const & a, T const & b) {
 return a > b ? a : b;
}
```

宏的另一个问题是名称不能重载，或者（把名称）限制在作用域中。
无论是在作用域，还是名字空间中，宏都将干扰具有相同名称的任何函数或变量。
因此，为宏使用足够长且唯一的名称非常重要，尤其是在头文件中。

### 使用`fastcall`函数

- 在32位模式下，关键字`__fastcall`更改函数调用方法，以便前两个（CodeGear编译器上是三个）整型参数使用寄存器传输，而不是用堆栈传输。
这可以提高具有整型参数函数的速度。浮点参数不受`__fastcall`的影响。
类的成员函数中的隐式“this”指针也被视为参数，因此可能只剩下一个空闲寄存器来传输其他参数。
因此，在使用`__fastcall`时，请确保最关键的整型参数首先出现在函数参数中。
- 在64位模式下，函数参数默认传输到寄存器中。 因此，在64位模式下无法识别`__fastcall`关键字。


### 函数本地化

仅在同一模块（即当前.cpp文件）中使用的函数应该被设置为本地函数。
这使编译器更容易生成内联函数，并跨函数间进行优化。
有三种方法可以使函数本地化：

1. 将关键字static添加到函数声明中。这是最简单的方法，但它不适用于类成员函数，其中static具有不同的含义。

2. 将函数或类放入匿名的名字空间。

3. Gnu编译器允许使用`"__attribute__((visibility("hidden")))"`.

### 使用整个程序优化

有些编译器有编译选项用于整个程序优化，或者将多个.cpp文件组合到一个目标文件中。
这使编译器能够优化组成程序的所有.cpp模块的寄存器分配和参数传输。
整个程序优化不能用于函数库（以目标文件或库文件的方式分发）。

### 使用64位模式

**参数传输在64位模式下比在32位模式下更高效，在64位Linux中比在64位Windows中更高效。**
- 在64位Linux中，前六个整数参数和前八个浮点参数在寄存器中传输，总计最多十四个寄存器参数。
- 在64位Windows中，前四个参数在寄存器中传输，无论它们是整数还是浮点数。

因此，如果函数具有四个以上的参数，则64位Linux比64位Windows更有效。
在参数传递这方面，32位Linux和32位Windows之间没有区别。


### 7.15 函数参数

在大多数情况下，函数参数按值传输。这意味着参数的值将复制到局部变量。
这对于简单类型（如int，float，double，bool，enum以及指针和引用）都很有效。

数组总是作为指针传递，除非它们被封装到类或结构中。

如果参数具有复合类型（例如结构或类），则情况会更复杂。
如果满足以下所有条件，则复合类型参数的传输效率可以很高：
- 对象足够小，可以装进单个寄存器
- 对象没有复制构造函数，也没有析构函数
- 对象没有虚拟成员（虚拟函数）
- 对象不使用运行时类型标识（RTTI）

如果上述某一或某些条件不满足，则将使用指向对象的指针或引用来传递对象通常会更快。
如果对象很大，那么复制整个对象显然需要时间。
- 将对象复制到参数时，必须调用复制构造函数，
- 并且函数返回之前，如果有析构函数的话，还必须调用析构函数

将复合对象传递给函数的**首选方法是使用const引用**。
const引用确保不修改原始对象。
与指针或非const引用不同，const引用允许函数参数是表达式或匿名对象。
如果函数被内联，编译器可以轻松地优化掉const引用。

另一种方法是使该函数成为对象类或结构的成员。
这种方法效率同等的高。

简单的函数参数
- 在32位系统中传输到堆栈中，
- 但在64位系统的寄存器中传输。

后者更有效率。
- 64位Windows允许在寄存器中传输最多四个参数。
- 64位Unix系统允许在寄存器中传输多达14个参数（8个浮点数或2个加6个整数，指针或引用参数）。

成员函数中的this指针也计入一个参数。进一步的细节在手册5中给出：“不同的C++编译器和操作系统调用约定”。

### 7.16 函数返回类型

函数的返回类型最好是
- 简单类型
- 指针
- 引用
- 或void

返回复合类型的对象更复杂，并且通常效率低下。

对于要返回复合类型对象的情况，只有在该复合类型是最简单的情况，才能在寄存器中返回。
有关何时可以在寄存器中返回对象的详细信息，请参见手册5：“调用不同C++编译器和操作系统的约定”。

除最简单的情况外，复合对象还有一种返回方法，即把它们复制到一个位置，该位置由调用者使用一个隐藏的指针来指定。
如果有拷贝构造函数的话，通常会在此时被调用。当原始的（被拷贝的）对象被销毁时，析构函数也会被调用。
在简单的情况下，编译器可以避免对复制构造函数和析构函数的调用，方法是直接在最终目标上构造对象来，但不要指望它。

（译者注：这大概是指现在编译器基本都支持的返回值优化return value optimization吧。）

您可以考虑以下备选方案，而不是返回复合对象：

- 使该函数成为该对象的构造函数。

- 使该函数修改现有对象而不是创建新对象。
现有对象可以通过指针或引用传递给该函数，或者让该函数成为此对象类的成员。

- 让该函数返回指针或者引用，该指针或引用指向函数内定义的静态对象。
这种方法效率高，但有风险。
返回的指针或引用仅在下次调用函数前有效，此对象也可能被在不同的线程中改写。
如果您忘记将此本地对象设置为静态，则只要函数返回它就会立即失效。

- 让函数使用`new`操作构造一个对象，并返回指向它的指针。
由于动态内存分配的成本，这种方法效率较低。如果您忘记删除对象，此方法还会导致内存泄漏的风险。

### 7.17 Function tail calls

A tail call is a way of optimizing function calls. If the last statement of a function is a call to
another function, then the compiler can replace the call by a jump to the second function. An
optimizing compiler will do this automatically. The second function will not return to the first
function, but directly to the place where the first function was called from. This is more
efficient because it eliminates a return. Example:

```cpp
// Example 7.35. Tail call
void function2(int x);
void function1(int y) {
   ...
   function2(y+1);
}
```

Here, the return from function1 is eliminated by jumping directly to function2. This
works even if there is a return value:

```cpp
// Example 7.36. Tail call with return value
int function2(int x);
int function1(int y) {
   ...
   return function2(y+1);
}
```

The tail call optimization works only if the two functions have the same return type. If the
functions have parameters on the stack (which is mostly the case in 32-bit mode) then the
two functions must use the same amount of stack space for parameters.

### 7.18 Recursive functions

A recursive function is a function that calls itself. Recursive function calls can be useful for
handling recursive data structures. The cost of recursive functions is that all parameters and
local variables get a new instance for every recursion, and this takes up stack space. Deep
recursions also makes the prediction of return addresses less efficient. This problem
typically appears with recursion levels deeper than 16 (see the explanation of return stack
buffer in manual 3: "The microarchitecture of Intel, AMD and VIA CPUs").

Recursive function calls can still be the most efficient solution for handling a branching data
tree structure. Recursion is more efficient if the tree structure is broad than if it is deep. A
non-branching recursion can always be replaced by a loop, which is more efficient. A
common textbook example of a recursive function is the factorial function:

```cpp
// Example 7.37. Factorial as recursive function
unsigned long int factorial(unsigned int n) {
   if (n < 2) return 1;
   return n * factorial(n-1);
}
```

This implementation is very inefficient because all the instances of n and all the return
addresses take up storage space on the stack. It is more efficient to use a loop:

```cpp
// Example 7.38. Factorial function as loop
unsigned long int factorial(unsigned int n) {
   unsigned long int product = 1;
   while (n > 1) {
   product *= n;
   n--;
   }
   return product;
}
```

Recursive tail calls are more efficient than other recursive calls, but still less efficient than a loop. 

Novice programmers sometimes make a call to `main` in order to restart their program. This
is a bad idea because the stack becomes filled up with new instances of all local variables
for every recursive call to `main`. The proper way to restart a program is to make a loop in
`main`.